In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
import seaborn as sns
import plotly.express as px
import os
import missingno as msno
import pickle
from glob import glob
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")
import matplotlib
import random

from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import RobustScaler

import tensorflow as tf
from keras.layers import *
from keras import *
import keras

In [ ]:
orders = pd.read_csv('orders.csv')

def beg_end_month(x):
    if x<=10:
        return '월초'
    elif 10<x<=20:
        return '월중'
    elif 20<x<=31:
        return '월말'
orders.BKG_DATE = pd.to_datetime(orders.BKG_DATE, format='%Y-%m-%d')
orders.INS_DATE = pd.to_datetime(orders.INS_DATE, format='%Y-%m-%d')
orders["BKG_TIME"] = pd.to_datetime(orders["BKG_TIME"], format='%Y-%m-%d %H:%M:%S')
orders['BKG_WEEK'] = orders.BKG_DATE.dt.week
orders['BKG_MONTH2'] = orders.BKG_DATE.dt.day.map(beg_end_month)

In [ ]:
data = orders[orders.BKG_TYP==7][orders.CORP_ID=='KX007'].groupby(['BKG_DATE','BKG_HOUR'])['ITEM_QTY'].sum().reset_index()
data = data.append(pd.DataFrame(dict(zip(['BKG_DATE','BKG_HOUR','ITEM_QTY'],[(pd.to_datetime('2021-06-28'),pd.to_datetime('2021-06-28')), (4,5), (0,0)]))))
data = data.sort_values(['BKG_DATE','BKG_HOUR'])

comb_date = []
for date, hour, _ in data.values:
    comb_date.append(pd.to_datetime(f'{date.year}-{date.month}-{date.day} {hour}:00:00'))
data['DATE'] = comb_date
data = data.set_index('DATE').drop(columns=['BKG_DATE', 'BKG_HOUR']).rename(columns={'ITEM_QTY':'TARGET'})

In [ ]:
nestle = ['2021-03-22', '2021-04-15', '2021-05-17', '2021-06-17']
nut_one = ['2021-04-25', '2021-04-30','2021-05-16','2021-05-31',
           '2021-06-13', '2021-06-27']

lg3 = [f'2021-03-{i}' for i in range(22,29)]
lg6 = [f'2021-06-{i}' for i in range(7,16)]
lg_brand = ['2021-04-20','2021-05-17']
lg = lg3 + lg_brand + lg6

data['nestle'] = np.zeros((data.shape[0],1))
data['nut_one'] = np.zeros((data.shape[0],1))
data['lg'] = np.zeros((data.shape[0],1))
for day in nestle:
    data.loc[day,'nestle'] = 1
for day in nut_one:
    data.loc[day,'nut_one'] = 1
for day in lg:
    data.loc[day,'lg'] = 1

In [ ]:
21*24+17

In [ ]:
data = data2.TARGET[-521-25:-17]

In [ ]:
((data.shape[0]-(input_window+output_window))//window)-24

In [ ]:
std_day = 21
input_window = 24*std_day
output_window = 1
window = 1
num_features = 4

test_window_x = np.zeros((((data.shape[0]-(input_window+output_window))//window)-24, input_window, num_features)) # batch_size, input, feature
for start in range(0,(data.shape[0]-(input_window+output_window))-24, window):
    end = start+input_window
    test_window_x[start//window,:,:] = data[start:end,:]

In [ ]:
data.TARGET[-545:-17].reset_index().to_csv('data.csv', index=False)

In [ ]:
np.save('data.npy', .values)

In [ ]:
scaler = RobustScaler()
train = scaler.fit_transform(data2.TARGET.values.reshape(-1,1))

In [ ]:
data2 = data.copy()

train_y = data2.iloc[:-720].values
val_y = data2.iloc[-720:-360].values
test_y = data2.iloc[-360:].values

val_index = data2.iloc[-720:-360].index
test_index = data2.iloc[-360:].index



In [ ]:
data2.TARGET.values.reshape(-1,1)

In [ ]:
with open('robust_scaler.pickle','wb') as f:
    pickle.dump(scaler, f)

In [ ]:
data2[-24:]

In [ ]:
data2 = data.copy()

train_y = data2.iloc[:-720].values
val_y = data2.iloc[-720:-360].values
test_y = data2.iloc[-360:].values

val_index = data2.iloc[-720:-360].index
test_index = data2.iloc[-360:].index

scaler = RobustScaler()
train = scaler.fit_transform(train_y)
val = scaler.transform(val_y)
test = scaler.transform(test_y)

train_y = data2.iloc[:-720,0].values.reshape(-1,1)
val_y = data2.iloc[-720:-360,0].values.reshape(-1,1)
test_y = data2.iloc[-360:,0].values.reshape(-1,1)

In [ ]:
std_day = 21
input_window = 24*std_day
output_window = 1
window = 1
num_features = 4

train_window_x = np.zeros((((train.shape[0]-(input_window+output_window))//window)-24, input_window, num_features)) # batch_size, input, feature
train_window_y = np.zeros((((train.shape[0]-(input_window+output_window))//window)-24, output_window, 1))
for start in range(0,(train.shape[0]-(input_window+output_window))-24, window):
    end = start+input_window
    train_window_x[start//window,:,:] = train[start:end,:]
    train_window_y[start//window,:] = train_y[end+24:end+output_window+24,:]

val_window_x = np.zeros((val.shape[0], input_window, num_features)) # batch_size, input, feature
for start in range(0,val.shape[0], window):
    end = start+input_window
    val_window_x[start//window,:,:] = np.concatenate([train[-24*std_day:],val])[start:end,:]

test_window_x = np.zeros((((test.shape[0], input_window, num_features)))) # batch_size, input, feature
for start in range(0, test.shape[0], window):
    end = start+input_window
    test_window_x[start//window,:,:] = np.concatenate([np.concatenate([train[(15-24)*std_day:],val])[-24*std_day:],test])[start:end,:]

In [ ]:
tf.random.set_seed(99)
np.random.seed(11)
random.seed(12)

model = Sequential(
    [LSTM(96, return_sequences=True),
    LayerNormalization(),
    LSTM(96, return_sequences=False),
    LayerNormalization(),
    Dense(1)]
    )

model.compile(optimizer='adam', loss='mse', metrics=[keras.metrics.RootMeanSquaredError()])

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50)
history = model.fit(train_window_x, train_window_y, epochs=500, batch_size=64, validation_data = (val_window_x,val_y), callbacks=[tf.keras.callbacks.ModelCheckpoint(filepath='model.keras',monitor='val_loss',mode='min',save_best_only=True)])

In [ ]:
plt.figure(figsize=(10,4))
plt.plot(history.history['root_mean_squared_error'], label='train_loss')
plt.plot(history.history['val_root_mean_squared_error'], label='test_loss')
plt.vlines(np.argmin(history.history['root_mean_squared_error']),0,1200, colors='red')
plt.vlines(np.argmin(history.history['val_root_mean_squared_error']),0,1200, colors='red')
plt.text(np.argmin(history.history['root_mean_squared_error']),0, 'train'+str(np.argmin(history.history['root_mean_squared_error'])+1), horizontalalignment='center')
plt.text(np.argmin(history.history['val_root_mean_squared_error']),0, 'val'+str(np.argmin(history.history['val_root_mean_squared_error'])+1), horizontalalignment='center')
plt.legend()
plt.show()

In [ ]:
test_window_x = np.zeros((((test.shape[0], input_window, num_features)))) # batch_size, input, feature
for start in range(0, test.shape[0], window):
    end = start+input_window
    test_window_x[start//window,:,:] = np.concatenate([np.concatenate([train[(15-24)*std_day:],val])[-24*std_day:],test])[start:end,:]

prediction = model.predict(test_window_x)

np.sqrt(mean_squared_error(prediction,test_y))

In [ ]:
fig, ax = plt.subplots(2,1,figsize=(40,10))
ax[0].plot(val_index, val_y, 'pink', label='real')
ax[0].plot(val_index, model.predict(val_window_x), 'k', label='pred')
ax[0].set_title('Validation Set')
ax[0].set_xticks(val_index)
ax[0].set_xticklabels(val_index,rotation=90, fontsize=7)
ax[0].set_xlim(val_index.min(),val_index.max())
ax[0].grid()
ax[0].legend()
ax[1].plot(test_index, test_y, 'pink', label='real')
ax[1].plot(test_index, prediction, 'k', label='pred')
ax[1].set_title('Test Set')
ax[1].set_xticks(test_index)
ax[1].set_xticklabels(test_index,rotation=90, fontsize=7)
ax[1].set_xlim(test_index.min(),test_index.max())
ax[1].grid()
ax[1].legend()
plt.tight_layout()
plt.show()

In [ ]:
model.save('lstm_9696.keras')

In [ ]:
tf.random.set_seed(99)
np.random.seed(11)
random.seed(12)

model = Sequential(
    [GRU(64, return_sequences=True),
    LayerNormalization(),
    GRU(96, return_sequences=False),
    LayerNormalization(),
    Dense(1)]
    )

model.compile(optimizer='adam', loss='mse', metrics=[keras.metrics.RootMeanSquaredError()])

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50)
history = model.fit(train_window_x, train_window_y, epochs=500, batch_size=64, validation_data = (val_window_x,val_y), callbacks=[tf.keras.callbacks.ModelCheckpoint(filepath='model.keras',monitor='val_loss',mode='min',save_best_only=True)])

In [ ]:
plt.figure(figsize=(10,4))
plt.plot(history.history['root_mean_squared_error'], label='train_loss')
plt.plot(history.history['val_root_mean_squared_error'], label='test_loss')
plt.vlines(np.argmin(history.history['root_mean_squared_error']),0,1200, colors='red')
plt.vlines(np.argmin(history.history['val_root_mean_squared_error']),0,1200, colors='red')
plt.text(np.argmin(history.history['root_mean_squared_error']),0, 'train'+str(np.argmin(history.history['root_mean_squared_error'])+1), horizontalalignment='center')
plt.text(np.argmin(history.history['val_root_mean_squared_error']),0, 'val'+str(np.argmin(history.history['val_root_mean_squared_error'])+1), horizontalalignment='center')
plt.legend()
plt.show()

In [ ]:
test_window_x = np.zeros((((test.shape[0], input_window, num_features)))) # batch_size, input, feature
for start in range(0, test.shape[0], window):
    end = start+input_window
    test_window_x[start//window,:,:] = np.concatenate([np.concatenate([train[(15-24)*std_day:],val])[-24*std_day:],test])[start:end,:]

prediction = model.predict(test_window_x)

np.sqrt(mean_squared_error(prediction,test_y))

In [ ]:
fig, ax = plt.subplots(2,1,figsize=(40,10))
ax[0].plot(val_index, val_y, 'pink', label='real')
ax[0].plot(val_index, model.predict(val_window_x), 'k', label='pred')
ax[0].set_title('Validation Set')
ax[0].set_xticks(val_index)
ax[0].set_xticklabels(val_index,rotation=90, fontsize=7)
ax[0].set_xlim(val_index.min(),val_index.max())
ax[0].grid()
ax[0].legend()
ax[1].plot(test_index, test_y, 'pink', label='real')
ax[1].plot(test_index, prediction, 'k', label='pred')
ax[1].set_title('Test Set')
ax[1].set_xticks(test_index)
ax[1].set_xticklabels(test_index,rotation=90, fontsize=7)
ax[1].set_xlim(test_index.min(),test_index.max())
ax[1].grid()
ax[1].legend()
plt.tight_layout()
plt.show()

In [ ]:
model.save('gru_6496.keras')

----------------------------------------------------------------

In [ ]:
model_lstm = keras.models.load_model('lstm_9696.keras')
model_gru = keras.models.load_model('gru_6496.keras')

In [ ]:
prediction_test_lstm = model_lstm.predict(test_window_x)

np.sqrt(mean_squared_error(prediction_test_lstm,test_y))

In [ ]:
prediction_test_gru = model_gru.predict(test_window_x)

np.sqrt(mean_squared_error(prediction_test_gru,test_y))

In [ ]:
prediction_test_ensemble = (prediction_test_lstm +  prediction_test_gru)/2

np.sqrt(mean_squared_error(prediction_test_ensemble, test_y))

In [ ]:
fig, ax = plt.subplots(3,1,figsize=(40,15))
ax[0].plot(test_index, test_y, 'pink', label='real')
ax[0].plot(test_index, prediction_test_lstm, 'k', label='pred')
ax[0].set_title('LSTM-Test Set')
ax[0].set_xticks(test_index)
ax[0].set_xticklabels(test_index,rotation=90, fontsize=7)
ax[0].set_xlim(test_index.min(),test_index.max())
ax[0].grid()
ax[0].legend()
ax[1].plot(test_index, test_y, 'pink', label='real')
ax[1].plot(test_index, prediction_test_gru, 'k', label='pred')
ax[1].set_title('GRU-Test Set')
ax[1].set_xticks(test_index)
ax[1].set_xticklabels(test_index,rotation=90, fontsize=7)
ax[1].set_xlim(test_index.min(),test_index.max())
ax[1].grid()
ax[1].legend()
ax[2].plot(test_index, test_y, 'pink', label='real')
ax[2].plot(test_index, prediction_test_ensemble, 'k', label='pred')
ax[2].set_title('Ensemble LSTM&GRU-Test Set')
ax[2].set_xticks(test_index)
ax[2].set_xticklabels(test_index,rotation=90, fontsize=7)
ax[2].set_xlim(test_index.min(),test_index.max())
ax[2].grid()
ax[2].legend()
plt.tight_layout()
plt.show()

In [ ]:
pred_val = np.concatenate([prediction_val_lstm, prediction_val_gru, prediction_val_ensemble], axis=1)
pred_test = np.concatenate([prediction_test_lstm, prediction_test_gru, prediction_test_ensemble], axis=1)
pred_val = pd.DataFrame(pred_val, columns=['lstm','gru','ensemble']).set_index(val_index.values).reset_index()
pred_test = pd.DataFrame(pred_test, columns=['lstm','gru','ensemble']).set_index(test_index.values).reset_index()
results = pd.concat([pred_val, pred_test]).reset_index(drop=True).rename(columns={'index':'DATE'})

In [ ]:
results.to_csv('results_dnn.csv', index=False)